Installing requirements

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00


Importing some requirements

In [2]:
import pandas as pd
import bs4
from sklearn import preprocessing, model_selection, metrics
import datasets
import transformers
import evaluate
import numpy as np

Configs

In [5]:
data_path = "/content/drive/MyDrive/spamtextclassifier/dataset.csv" #@param {type:"string"}
text_column_name = "email" #@param {type:"string"}
label_column_name = "category" #@param {type:"string"}

model_name = "distilbert-base-uncased" #@param {type:"string"}
test_size = 0.2 #@param {type:"number"}
num_labels = 2 #@param {type:"number"}

Reading and cleaning the dataset

In [6]:
df = pd.read_csv(data_path, header=0)
df.head()

,email,category
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam
4,"I attended the same conference, and was impres...",not-spam


In [7]:
def clean(text):
  text = text.replace("<p>", "</p>\n")
  text = bs4.BeautifulSoup(text, "lxml").text
  return text

df['cleaned_text'] = df[text_column_name].apply(clean)

<ipython-input-7-bdc7ff10e4a4>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = bs4.BeautifulSoup(text, "lxml").text


Label Encoder

In [8]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df[label_column_name].tolist())
df.head()

,email,category,cleaned_text,label
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam,"URL: http://www.newsisfree.com/click/-1,817167...",0
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",0
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam,"Dan Kohn writes:\n\n\n\n> Guys, the Habeas In...",0
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam,wintermute wrote:\n\n>>Anyone know where in Ir...,0
4,"I attended the same conference, and was impres...",not-spam,"I attended the same conference, and was impres...",0


Training/Testing split

In [9]:
df_train, df_test = model_selection.train_test_split(df, test_size=test_size)

Convert dataset to Huggingface dataset

In [10]:
train_dataset = datasets.Dataset.from_pandas(df_train)
test_dataset = datasets.Dataset.from_pandas(df_test)

Tokenizer

In [11]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
def preprocess_funct(examples):
  return tokenizer(examples["cleaned_text"], truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
tokenized_train = train_dataset.map(preprocess_funct, batched=True)
tokenized_test = test_dataset.map(preprocess_funct, batched=True)

Map:   0%|          | 0/3036 [00:00<?, ? examples/s]

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

Initialising and Training Model

In [13]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
data_collator = transformers.DataCollatorWithPadding(tokenizer = tokenizer)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis = -1)
  return metric.compute(predictions = predictions, references = labels)

training_args = transformers.TrainingArguments(
    output_dir = "./results",
    learning_rate = 2e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 5,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    logging_strategy = "epoch",
    report_to="none"
)

trainer = transformers.Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    processing_class = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()
trainer.save_model("spam_model")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.200600,0.136624,0.963158
2,0.085500,0.086100,0.972368
3,0.028600,0.069573,0.988158
4,0.022900,0.057709,0.992105
5,0.003300,0.048969,0.993421


Evaluate Model

In [15]:
print("Evaluation for training set:")
predictions = trainer.predict(tokenized_train)
predictions = np.argmax(predictions[:3][0], axis = 1)
GT = df_train['label'].tolist()
print(metrics.classification_report(GT, predictions))

Evaluation for training set:


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1528
           1       1.00      1.00      1.00      1508

    accuracy                           1.00      3036
   macro avg       1.00      1.00      1.00      3036
weighted avg       1.00      1.00      1.00      3036



In [18]:
print("Evaluation for testing set:")
predictions = trainer.predict(tokenized_test)
predictions = np.argmax(predictions[:3][0], axis = 1)
GT = df_test['label'].tolist()
print(metrics.classification_report(GT, predictions))

Evaluation for testing set:


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       372
           1       0.99      0.99      0.99       388

    accuracy                           0.99       760
   macro avg       0.99      0.99      0.99       760
weighted avg       0.99      0.99      0.99       760



Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
!mv /content/results /content/drive/MyDrive/spamtextclassifier/results

In [17]:
!mv /content/spam_model /content/drive/MyDrive/spamtextclassifier/spam_model